# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [12]:
#Libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


In [13]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [14]:
# Separar características y variable objetivo
y = spaceship['Transported']  # Variable objetivo
X = spaceship.drop(columns=['Transported'])  # Características

# Identificar variables numéricas y categóricas
num_features = X.select_dtypes(include=['int64', 'float64']).columns
cat_features = X.select_dtypes(include=['object']).columns

# Pipelines para preprocesamiento
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Transformador de columnas
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

# Aplicar preprocesamiento
X_processed = preprocessor.fit_transform(X)

# Selección de características
selector = SelectKBest(score_func=f_classif, k=10)  # Selecciona las 10 mejores características
X_selected = selector.fit_transform(X_processed, y)


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

- Evaluate your model

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

- Run Grid Search

- Evaluate your model

In [ ]:
# Separar características y variable objetivo
y = spaceship['Transported']  # Variable objetivo
X = spaceship.drop(columns=['Transported'])  # Características

# Identificar variables numéricas y categóricas
num_features = X.select_dtypes(include=['int64', 'float64']).columns
cat_features = X.select_dtypes(include=['object']).columns

# Pipelines para preprocesamiento
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Transformador de columnas
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

# Aplicar preprocesamiento
X_processed = preprocessor.fit_transform(X)

# Selección de características
selector = SelectKBest(score_func=f_classif, k=10)  # Selecciona las 10 mejores características
X_selected = selector.fit_transform(X_processed, y)

# Definir el modelo base
base_model = DecisionTreeClassifier(random_state=42)

# Definir el modelo Bagging
bagging_model = BaggingClassifier(estimator=base_model, random_state=42)

# Definir los hiperparámetros para ajuste
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_samples': [0.5, 0.7, 1.0],
    'max_features': [0.5, 0.7, 1.0]
}

# Grid Search para encontrar la mejor combinación
grid_search = GridSearchCV(bagging_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_selected, y)

# Mejor modelo
best_bagging = grid_search.best_estimator_

# Evaluación del mejor modelo
y_pred_best = best_bagging.predict(X_selected)
best_accuracy = accuracy_score(y, y_pred_best)

print(f"Mejor exactitud del modelo Bagging después del ajuste de hiperparámetros: {best_accuracy}")
